--- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

30373
25512
65332
33549
35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [1]:
import scala.io.Source

val filename = "day8input"

val rawData = Source.fromFile(filename).getLines.toList

import scala.io.Source


filename: String = "day8input"
rawData: List[String] = List(
  "002100030222341303232115221524311502200403234452204125251345024423544443305142410110142101223120110",
  "102030322341143330244233013052025453422005415304655426206452251310214055314024034341233110302320003",
  "003012203404011430120134554314552441635001641050211406204621114102021543144514102510421243314102033",
  "330313423002034140035001210441156554545625042262651356556544202500446121455112434243313013430141203",
  "331203121131230002055113430244610062654034520106363462155201210355601153542301323442223310134324300",
  "232201231021003254355411402310326206625011564114125202263504451451212600155351424154310240411213431",
  "033033333301011523544023206136145505304224325606020360006206455564513553512054354124523232201230441",
  "132201032234334044251334220345360402456541662253556375754662323265201641514121105025300453041111234",
  "31432234132210440223502020656504422536352761776757452447542176350434106

In [2]:
val columns = rawData.head.length
val rows = rawData.length

columns: Int = 99
rows: Int = 99

In [3]:
val treeMap = Array.ofDim[Int](rows, columns)

rawData.zipWithIndex.foreach{
    case (rowData, row) => rowData.toList.zipWithIndex.foreach{
        case (tree, column) => treeMap(row)(column) = tree.toString.toInt
    }
}

treeMap: Array[Array[Int]] = Array(
  Array(
    0,
    0,
    2,
    1,
    0,
    0,
    0,
    3,
    0,
    2,
    2,
    2,
    3,
    4,
    1,
    3,
    0,
    3,
    2,
    3,
    2,
    1,
    1,
    5,
    2,
    2,
    1,
    5,
    2,
    4,
    3,
    1,
    1,
    5,
    0,
    2,
    2,
...

In [4]:
def getColumn(column: Int, a: Array[Array[Int]]): Array[Int] = a.map(_(column))

defined function getColumn

In [5]:
def getTreeHeights(rung: Array[Int], slice: (Int, Int)): List[Int] = {
    rung.slice(slice._1, slice._2).toList
}

def getHeightsInDirection(coords: (Int, Int), map: Array[Array[Int]]) = {
    val (row, column) = coords
    val rowData = map(row)
    val columnData = getColumn(column, map)
    List(
        getTreeHeights(rowData, (0, column)).reverse,
        getTreeHeights(rowData, (column + 1, columns)),
        getTreeHeights(columnData, (0, row)).reverse,
        getTreeHeights(columnData, (row + 1, rows)),
    )
}

def isTreeVisable(coords: (Int, Int), map: Array[Array[Int]]): Boolean = {
    val tree = map(coords._1)(coords._2)
    val heights = getHeightsInDirection(coords, map)
    heights.map(_.max).map(_ < tree).reduce((left, right) => left || right)
}

defined function getTreeHeights
defined function getHeightsInDirection
defined function isTreeVisable

In [6]:
val visableMap = Array.ofDim[Boolean](rows, columns)

val maxRowIndex = rows - 1
val maxColumnIndex = columns -1

for (i<-0 until rows; j<-0 until columns) {
    val coords = (i, j)
    visableMap(i)(j) = coords match {
        case (0, _) | (_, 0) | (98, _) | (_, 98) => true
        case _ => isTreeVisable(coords, treeMap)
    }
}

visableMap: Array[Array[Boolean]] = Array(
  Array(
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
    true,
...
maxRowIndex: Int = 98
maxColumnIndex: Int = 98

In [7]:
visableMap.flatten.toList.count(x => x)

res6: Int = 1698

--- Part Two ---

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

30373
25512
65332
33549
35390

    Looking up, its view is not blocked; it can see 1 tree (of height 3).
    Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
    Looking right, its view is not blocked; it can see 2 trees.
    Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

30373
25512
65332
33549
35390

    Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
    Looking left, its view is not blocked; it can see 2 trees.
    Looking down, its view is also not blocked; it can see 1 tree.
    Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?


In [8]:
def getScenicScore(coords: (Int, Int), map: Array[Array[Int]]): Int = {
    val tree = map(coords._1)(coords._2)
    val heights = getHeightsInDirection(coords, map)
    val treesNotTaller = heights.map(_.span(_ < tree)._1)
    val scores = treesNotTaller.zipWithIndex.map{
        case (notTaller, index) => if (heights(index).size != 0 && notTaller.isEmpty) {
            1
        } else if (heights(index).size == 0) {
            0
        } else if (heights(index).size == notTaller.size) {
            notTaller.size
        } else {
            notTaller.size + 1
        }
    }
    scores.reduce((left, right) => left * right)
}

defined function getScenicScore

In [9]:
val scenicMap = Array.ofDim[Int](rows, columns)

val maxRowIndex = rows - 1
val maxColumnIndex = columns -1

for (i<-0 until rows; j<-0 until columns) {
    val coords = (i, j)
    scenicMap(i)(j) = getScenicScore(coords, treeMap)
}

scenicMap: Array[Array[Int]] = Array(
  Array(
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
...
maxRowIndex: Int = 98
maxColumnIndex: Int = 98

In [11]:
scenicMap.flatten.toList.max

res10: Int = 672280